# SDK Examples - Cookbooks

Examples for new Cookbooks functions that were released with smsdk v1.1

*created April 2023*

In [ ]:
from smsdk import client
from datetime import datetime, timedelta
import pandas as pd
import os

In [ ]:
env_var_tenant = 'ENV_SDK_VAR_TENANT'
env_var_api_key = 'ENV_SDK_VAR_API_KEY'
env_var_api_secret = 'ENV_SDK_VAR_API_SECRET'

# Check if the environment variable exists
if env_var_tenant in os.environ:
    # Retrieve the value of the environment variable
    tenant = os.environ[env_var_tenant]
else:
    # Use a default value if the environment variable is not present
    tenant = 'demo-continuous'

# Check if the environment variable exists
if env_var_api_key in os.environ:
    # Retrieve the value of the environment variable
    api_key = os.environ[env_var_api_key]
else:
    # Use a default value if the environment variable is not present
    api_key = ''

# Check if the environment variable exists
if env_var_api_secret in os.environ:
    # Retrieve the value of the environment variable
    api_secret = os.environ[env_var_api_secret]
else:
    # Use a default value if the environment variable is not present
    api_secret = ''

In [ ]:
cli = client.Client(tenant)
cli.login('apikey', 
          key_id = api_key, 
          secret_id = api_secret)

machine_types = cli.get_machine_type_names()
machine_types

# Cookbooks

The SDK has several functions related to cookbooks allowing you to do things like look at the configuration of cookbooks and see the top runs of various recipes.

## Get Cookbooks

```cli.get_cookbooks()```

Return a list of all cookbooks on this tenant, both deployed and undeployed. Cookbook data is in JSON format, which in python translates to dictionaries and lists.

Note: Recipe groups are reflected as "Products" in the UI.

In [ ]:
cookbooks = cli.get_cookbooks()

# example: count cookbooks
print("Total number of cookbooks:", len(cookbooks))

# example: basic info about all cookbooks
df_cookbooks = pd.DataFrame(cookbooks)
df_cookbooks[["name", "assetNames", "id"]]

In [ ]:
# example: check out data from one of the cookbooks
cookbook = cookbooks[2]
cookbook

In [ ]:
cookbook = cookbooks[2]
# set a specific recipe group (product) for the examples below
recipe_group_idx = 0

# example: list outcomes in this cookbook & product
print("Outcomes:")
for i in cookbook["recipe_groups"][recipe_group_idx]["outcomes"]:
    print(i["field"]["fieldName"], "- weight:", i["weight"])
print()
# example: list levers in this cookbook & product
print("Levers:")
for i in cookbook["recipe_groups"][recipe_group_idx]["levers"]:
    print(i["fieldName"])
print()
# example: list conditions/constraints in this cookbook & product
print("Constraints (Conditions):")
for i in cookbook["recipe_groups"][recipe_group_idx]["constraints"]:
    print(i["field"]["fieldName"])

# NOTE: you could also use the schema for this machine to convert these raw field names to display names

In [ ]:
cookbook = cookbooks[2]

# example: getting a list of all recipe group IDs (products) for a cookbook
print("Products:")
recipe_group_ids = [(i["id"],i["values"]) for i in cookbook["recipe_groups"]]
recipe_group_ids

## Get Top Runs/Results

```cli.get_cookbook_top_results(recipe_group_id, limit)```

This function returns the top runs for the selected recipe group (product). Set the limit as you would set the run limit in the Cookbooks UI (default = 10). **Note: the cookbook must be deployed for this to work. This means that not all cookbooks returned by cli.get_cookbooks can nessarily be used with cli.get_cookbook_top_results.** See [this](https://github.com/sightmachine/sightmachine-sdk/blob/master/docs/commonly_used_data_types/run.md) GitHub doc for more details on the format of the run data that's returned.

In [ ]:
cookbook = cookbooks[2]
recipe_group_ids = [i["id"] for i in cookbook["recipe_groups"]]
recipe_group_idx = 0

print("Cookbook name:", cookbook["name"])
print("Product:", cookbook["recipe_groups"][recipe_group_idx]["values"])

results = cli.get_cookbook_top_results(recipe_group_ids[recipe_group_idx], 10)

# extract the two parts of the run data - recipe view and run view (corresponding to two views in Cookbooks UI)
runs = results["runs"]
recipes = results["constraint_groups"]

In [ ]:
# check out a sample run
runs[0]

In [ ]:
# check out a sample recipe
recipes[0]

In [ ]:
# example: print some stats
print("Total number of runs for this recipe group (product):", len(runs))
unmuted_runs = [r for r in runs if r["_count"]>r["_count_muted"]]
print("Total number of runs with unmuted records:",len(unmuted_runs) )

In [ ]:
# example: make a runs table
df_runs = pd.DataFrame(runs)
df_runs

# prune this table down to get run-wise stats that could be graphed 

In [ ]:
# example: make a recipes table (matching cookbooks Recipes UI)
df_recipes = pd.DataFrame(recipes)
df_recipes

In [ ]:
# example: count number of constraint groups (recipes)
print("Total number of constraint groups (recipes) shown in final cookbook:", len(df_recipes.index))

Note that you could compute the recipes table yourself if you take the run table, group by constraint group (recipe), and aggregate several of the fields.

In [ ]:
print(df_runs.constraint_group_id.nunique() == len(df_recipes.index))

## Get Current Value

```cli.get_cookbook_current_value(variables, minutes)```

This function gets the most recently reported values of the tags passed into it. See the example for parameter formatting. 
- ```variables``` is a list of dictionaries which must have the keys 'asset' and 'name'.
    - Asset must be the official name, not the display name. It will be the same as the 'asset' values stored in the cookbook top runs data, or the values in the 'assetNames' list included in a cookbook's configuration.
    - Name must be the official tag name, not a tag display name. Tags used here can be any tag associated with the chosen asset **except for** KPI-computed tags/outcomes.
    - Be sure to get the asset name and tag name by looking at a sample run for your cookbook. That way, you can make sure the asset and tag name are correctly formatted. 
    - Note that if even one of the dicts in this list is invalid, the function will throw an error and neglect to tell you which parameter is causing the issue.
- ```minutes``` is optional and defines the number of minutes back you want to look (defaults to 1440 or one day). 
    - None is returned if the tag does not have any reported values in the last ```minutes``` number of minutes.

In [ ]:
# get the current value for a constraint and non-KPI outcome
vars = [{'asset': 'JB_HM_Diecast_1', 'name': "stats__AluminumTempAvg__val"},
        {'asset': 'JB_HM_Diecast_1', 'name': "stats__ProductSKU__val"}]
vals_dict = cli.get_cookbook_current_value(vars)
pd.DataFrame(vals_dict)

In [ ]:
# example: a setting for minutes that doesn't go far enough back to find last value
vals_dict = cli.get_cookbook_current_value(vars, minutes=0.5)
pd.DataFrame(vals_dict)

In [ ]:
# get the current value for a tag that's not necessarily in cookbook
vars = [{'asset': 'JB_HM_Diecast_1', 'name': "stats__AluminumTempAvg__val"}]
vals_dict = cli.get_cookbook_current_value(vars)
pd.DataFrame(vals_dict)

## Normalize Constraints

```cli.normalize_constraints(constraint_values)```

Return a clean string version of constraint fields. Helps turn the dictionary constraint definitions into a more concise string format. ```constraint_values``` must be a list of constraint dicts, each of which include at least a "to" and "from" key. This function only works on range-based constraints (i.e. when the condition tag is continuous, not categorical).

Example:
- input = ```[{'from': 120, 'from_is_inclusive': True, 'to': None, 'to_is_inclusive': False}, ...]```
- output = ```['[120,None)', ...]```

In [ ]:
if len(runs) and len(runs[0]["constraints"]):
    range_example = runs[0]["constraints"][0]["values"]
    print(range_example)

In [ ]:
# turn constraint dicts to strings for the first run
run_idx = 0
ranges = [i["values"] for i in runs[run_idx]["constraints"]]

print(ranges)
print(cli.normalize_constraints(ranges))

# NOTE not exactly the intended output - waiting on fix for this